In [1]:
import os
import copy
import glob

import numpy as np
import astropy.units as u
import astropy.constants as c
import astropy.coordinates as aco
import matplotlib.pyplot as plt

import hera_pspec as hp
from hera_pspec.data import DATA_PATH
from pyuvdata import UVData

In [2]:
# From Command Line Arguments
dfiles = sorted(glob.glob('/lustre/aoc/projects/hera/H1C_IDR2/IDR2_1/2458111/zen.2458111.?????.xx.HH.uvh5.OCRS'))
lst_range = (5.0, 6.0)
freq_chans = np.arange(580, 680 + 1)

In [3]:
%%time
uvd = UVData()
files = []
times = []
for dfile in dfiles:
    uvd.read_uvh5(dfile, read_data=False)
    LSTrads = np.unique(uvd.lst_array * u.rad)
    LSThours = aco.Angle(LSTrads).hour
    LSTindices = np.where(np.logical_and(LSThours >= lst_range[0], LSThours <= lst_range[-1]))[0]
    
    if LSTindices.size > 0:
        JDtimes = np.take(np.unique(uvd.time_array), LSTindices)
        files.append(dfile)
        times.append(JDtimes.tolist())

CPU times: user 835 ms, sys: 227 ms, total: 1.06 s
Wall time: 1.15 s


In [5]:
%%time
uvd = UVData()
uvd.read_uvh5(
    files[0],
    ant_str='cross',
    freq_chans=freq_chans,
    times=times[0])
for file, time in zip(files[1:], times[1:]):
    uvdi = UVData()
    uvdi.read_uvh5(
        file, 
        ant_str='cross',
        freq_chans=freq_chans,
        times=time)
    uvd += uvdi

CPU times: user 16.5 s, sys: 5.7 s, total: 22.2 s
Wall time: 22.1 s
